# Cleaning

In [10]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [11]:
data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/train_cleaned.csv")

data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_words
0,0000997932d777bf,explanation\n edits made username hardcore ...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '..."
1,000103f0d9cfb60f,daww matches background colour im seemingly ...,0,0,0,0,0,0,"['daww', 'matches', 'background', 'colour', 'i..."
2,000113f07ec002fd,hey man im really trying edit war guy c...,0,0,0,0,0,0,"['hey', 'man', 'im', 'really', 'trying', 'edit..."
3,0001b41b1c6bb37e,cant make real suggestions improvement won...,0,0,0,0,0,0,"['cant', 'make', 'real', 'suggestions', 'impro..."
4,0001d958c54c6e35,sir hero chance remember page thats,0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page', ..."


# Tokenizing Comments using TextVectorization

In [12]:
# finding the maximum number of words present in any given comment
maxlen = 0
longest_comment = ""
for comment in data['comment_text']:
    length = len(comment)
    if (length > maxlen):
        longest_comment = comment
    maxlen = max(maxlen, length)

print("Number of characters in the longest comment is", maxlen)
print("Number of words in the longest comment is",
      len(longest_comment.split(" ")) + 1)

Number of characters in the longest comment is 5000
Number of words in the longest comment is 456


In [14]:
import tensorflow as tf

vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(

    max_tokens=None,

    # this is greater than the max words any comment has (774)
    # the remaning spots in the output would be padded by 0s
    output_sequence_length=800,

    # converets to lowercase and skips all the punctuation
    standardize="lower_and_strip_punctuation",

    # the tokens will be split at whitespaces
    split="whitespace",

    # each of the tokens is represented as an integer
    output_mode="int",
)

In [15]:
numpyArray = data[data.columns[1]].to_numpy()
vectorize_layer.adapt(numpyArray)

In [65]:
# testing
vectorize_layer("hello, world!")

<tf.Tensor: shape=(800,), dtype=int64, numpy=
array([185, 161,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,  

# Creating a Tokenizer

### Gets all the words into one string

In [66]:
# Creates the tokenizer class
tokenizer = keras.preprocessing.text.Tokenizer()

# Combines all the words into one singular string
allWordsString = " ".join(data.head(50)["comment_text"].tolist())
allWordsList = allWordsString.split(r"\\s+")

# Updates the tokenizer with the string of all words
tokenizer.fit_on_texts(allWordsList)

# Prints word dictionary
# print(tokenizer.word_index)

# Prints length of word dictionary
print(len(tokenizer.word_index))

# Converts text to numbers
print(tokenizer.texts_to_sequences(["page", "im", "use", "mussolini"]))

1077
[[1], [2], [3], [1077]]


# Text Vectorization Layer
### Following [https://www.tensorflow.org/text/tutorials/text_classification_rnn](http://)

In [67]:
NUM_ROWS = 50000

MAX_LENGTH = None
encoder = tf.keras.layers.TextVectorization(output_sequence_length=MAX_LENGTH)
encoder.adapt(data.head(NUM_ROWS)["comment_text"].tolist())

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'article', 'page', 'wikipedia', 'talk', 'please',
       'would', 'one', 'like', 'dont', 'see', 'also', 'im', 'think',
       'know', 'edit', 'articles', 'people', 'use'], dtype='<U4955')

### Testing the encoder

Encoder removes punctuation and whitespace and forces lowercase so half the cleaning we did was useless

In [68]:
commentsToEncode = data.head(3)["comment_text"]
print(commentsToEncode)

encodedComments = encoder(commentsToEncode).numpy()
print(encodedComments)

for comment in encodedComments:
    print(" ".join(vocab[comment]))

0    explanation\n  edits made   username hardcore ...
1    daww  matches  background colour im seemingly ...
2    hey man im really  trying  edit war     guy  c...
Name: comment_text, dtype: object
[[  545    47    46   510  4851 10100   697   212  2036  9481  5882  2142
   2601    37   996 13902  2466     6    10   144   323     5     3    57
     13  3502]
 [97471  2159  1329  3969    13  4209  2214    23     5   899    94     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0]
 [  380   317    13    53   147    16   224   504  1980   387   376    27
    446    47   244     5     3   108   338  2144   540   331     0     0
      0     0]]
explanation edits made username hardcore metallica fan reverted werent vandalisms closure gas voted new york dolls fac please dont remove template talk page since im retired
daww matches background colour im seemingly stuck thanks talk january utc               
hey man im really trying edit war guy constantly

### Building the model

In [60]:
# Sets random seed so results are identical every time
SEED = 1
tf.random.set_seed(SEED)
np.random.seed(SEED)

from tensorflow.keras.callbacks import EarlyStopping

epochs = 25
learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
embedding_size = 512
lstm_size_1 = 128
conv_size_2 = 64

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=embedding_size,
        mask_zero=True,
        embeddings_regularizer=tf.keras.regularizers.L2(0.001)
    ),
    tf.keras.layers.Conv1D(128, 5, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(conv_size_2, 5, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_size_1, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

# Add early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])


### Training the model

In [63]:
binaryDf = data.head(NUM_ROWS)[["comment_text", "toxic"]]

split_cutoff = int(0.8 * NUM_ROWS)
training_data = binaryDf.iloc[:split_cutoff]
validation_data = binaryDf.iloc[split_cutoff:]

training_target = training_data.pop("toxic")
validation_target = validation_data.pop("toxic")

In [64]:
# # training_dataset = tf.convert_to_tensor(training_data)
# # validation_dataset = tf.convert_to_tensor(validation_data)

# validation_target = validation_data.pop("toxic")
# validation_dataset = tf.convert_to_tensor(validation_data)

# # history = model.fit(training_dataset, target, epochs=10, validation_data=(validation_dataset, validation_target))

# binaryDf = data.head(NUM_ROWS)[["comment_text", "toxic"]]
# target = binaryDf.pop("toxic")
# dataset = tf.convert_to_tensor(binaryDf)

# Early Stopping
callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=7)

history = model.fit(training_data, training_target, epochs=25, validation_data=(validation_data, validation_target), callbacks=[callback])

Epoch 1/25


/opt/conda/lib/python3.7/site-packages/keras/backend.py:5677: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits, "Sigmoid", "binary_crossentropy"


1250/1250 [==============================] - ETA: 0s - loss: 6.1844 - accuracy: 0.9088

/opt/conda/lib/python3.7/site-packages/keras/backend.py:5677: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits, "Sigmoid", "binary_crossentropy"


1250/1250 [==============================] - 64s 44ms/step - loss: 6.1844 - accuracy: 0.9088 - val_loss: 0.3019 - val_accuracy: 0.9459
Epoch 2/25
1250/1250 [==============================] - 45s 36ms/step - loss: 0.2551 - accuracy: 0.9551 - val_loss: 0.2366 - val_accuracy: 0.9546
Epoch 3/25
1250/1250 [==============================] - 45s 36ms/step - loss: 0.1877 - accuracy: 0.9676 - val_loss: 0.2077 - val_accuracy: 0.9576
Epoch 4/25
1250/1250 [==============================] - 44s 36ms/step - loss: 0.1466 - accuracy: 0.9763 - val_loss: 0.2014 - val_accuracy: 0.9538
Epoch 5/25
1250/1250 [==============================] - 46s 37ms/step - loss: 0.1197 - accuracy: 0.9806 - val_loss: 0.2397 - val_accuracy: 0.9553
Epoch 6/25
1250/1250 [==============================] - 45s 36ms/step - loss: 0.1006 - accuracy: 0.9864 - val_loss: 0.2138 - val_accuracy: 0.9542
Epoch 7/25
1250/1250 [==============================] - 45s 36ms/step - loss: 0.0861 - accuracy: 0.9886 - val_loss: 0.2445 - val_accura

In [1]:
# sample_text = ""
# predictions = model.predict(np.array([sample_text]))
# predictions[0]

In [69]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_4 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_7 (Embedding)     (None, None, 512)         55971840  
                                                                 
 conv1d_11 (Conv1D)          (None, None, 128)         327808    
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, None, 128)        0         
 g1D)                                                            
                                                                 
 conv1d_12 (Conv1D)          (None, None, 64)          41024     
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, None, 64)        

### Getting test data

In [70]:
test_data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test.csv")
test_labels = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test_labels.csv")

test_labels = test_labels.loc[test_labels["toxic"] >= 0]
merged_df = test_labels.merge(test_data, left_on="id", right_on="id")

In [71]:
# Tests all rows with a value of 0 or 1

test_df = merged_df[["comment_text", "toxic"]]
testTarget = test_df.pop("toxic")
model.evaluate(test_df, testTarget)

2000/2000 [==============================] - 32s 16ms/step - loss: 0.3612 - accuracy: 0.9078


[0.3611910939216614, 0.9077651500701904]

In [72]:
# Tests only rows with a toxic value of 1

test_df = merged_df[["comment_text", "toxic"]]
newTest_df = test_df.loc[test_df["toxic"] == 1]

newTestTarget = newTest_df.pop("toxic")
model.evaluate(newTest_df, newTestTarget)

191/191 [==============================] - 3s 16ms/step - loss: 1.1481 - accuracy: 0.7939


[1.148050308227539, 0.7939244508743286]